In [1]:
### XAI ----------------------------
# 버전 맞추기,,,,,,,
!pip install dice-ml
!pip install --force-reinstall numpy==1.23.5 pandas==1.5.3 scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you h

In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
import dice_ml
from dice_ml.utils import helpers
import xgboost as xgb
# 역변환
from scipy.special import inv_boxcox
### 모델링--------------------------
import xgboost as xgb
from xgboost import XGBClassifier
### 새로운 데이터 --------------------
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.preprocessing import LabelEncoder

#### 함수 정의

In [2]:
### 역변환!!!! - Counterfactual에서 사용
def inverse_scaling_boxcox(transformed_data, transformer_path, scaler_path):

    # 역변환 대상 변수
    cols = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수']
    cols_idx = [transformed_data.columns.get_loc(col) for col in cols]

    # scaler, transformer
    scaler = joblib.load(scaler_path)
    transformer = joblib.load(transformer_path)

    # 변환된 5개 변수만 추출
    transformed = transformed_data.iloc[:, cols_idx]

    # 역표준화
    un_scaled = scaler.inverse_transform(transformed)

    # 역변환
    un_transformed = np.array([inv_boxcox(un_scaled[:, i], transformer.lambdas_[i]) for i in range(un_scaled.shape[1])]).T

    # 전체 복원
    original = transformed_data.copy()
    for j, col in enumerate(cols):
        original[col] = un_transformed[:,j]


    ### 배수등급, 경사도 변수 처리
    # 경사도
    slope_encoding = {'0-2%': 0, '2-7%': 1, '7-15%': 2, '15-30%': 3, '30-60%': 4, '60-100%': 5}
    # 배수등급
    drain_encoding = {'매우양호': 5, '양호': 4, '약간양호': 3, '약간불량': 2, '불량': 1, '매우불량': 0}
    # 인코딩 딕셔너리 뒤집기
    slope_decoding = {v: k for k, v in slope_encoding.items()}
    drain_decoding = {v: k for k, v in drain_encoding.items()}

    # 역매핑
    original['경사도'] = original['경사도'].map(slope_decoding)
    original['배수등급'] = original['배수등급'].map(drain_decoding)

    return original

In [5]:
def dice(x_test, model_path, transformer_path, scaler_path): # x_test : 예측이 1로 나오는 예시 샘플 하나
    xgb_model = joblib.load(model_path)

    # 혹시 모르니까 한 번 더 변환해주고
    x_test = x_test.astype('float64')

    ### DiCE
    continuous = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수']
    d = dice_ml.Data(dataframe = x_test.assign(발생여부 = 0),  # dummy target
                     continuous_features = continuous,
                     outcome_name = "발생여부")
    m = dice_ml.Model(model = xgb_model, backend = "sklearn")
    exp = dice_ml.Dice(d, m)

    ### counterfactual 생성
    # 값을 바꿀 변수들만 지정
    features_to_change = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령']
    query_instance = x_test
    query_instance = query_instance.astype('float64')
    cf = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite",features_to_vary=features_to_change)

    ### 데이터프레임 저장
    # 기존 데이터
    original_one = cf.cf_examples_list[0].test_instance_df
    # 바뀐 데이터
    changed_zero = cf.cf_examples_list[0].final_cfs_df

    original_one = inverse_scaling_boxcox(original_one, transformer_path, scaler_path)
    changed_zero = inverse_scaling_boxcox(changed_zero, transformer_path, scaler_path)

    return original_one, changed_zero

#### Counterfactual 코드

In [ ]:
### Box-Cox 변환 - 기후 제외
# 변환 데이터
df = pd.read_csv('/content/drive/MyDrive/DATA_완성/dataset.csv')
# 변환 대상 컬럼
cols = ['승용차', '버스', '트럭',  '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수']
# 변환기, 스케일러
transform = PowerTransformer(method = 'box-cox', standardize = False)
scaler = StandardScaler()
# Box-Cox + 스케일링 진행
arr = df[cols].values + 1e-6
bc = transform.fit_transform(arr)
bc_std = scaler.fit_transform(bc)
df[cols] = bc_std
# 변환기, 스케일러 저장
joblib.dump(transform, '/content/drive/MyDrive/준희/Report/Files/BoxCoxTransformer_wo_env_dice.pkl')
joblib.dump(scaler, '/content/drive/MyDrive/준희/Report/Files/StandardScaler_wo_env_dice.pkl')

['/content/drive/MyDrive/준희/Report/Files/StandardScaler_wo_env_dice.pkl']

In [4]:
### model
model_path = '/content/drive/MyDrive/세은/model.pickle'
### transformer, scaler
transformer_path = '/content/drive/MyDrive/준희/Report/Files/BoxCoxTransformer_wo_env_dice.pkl'
scaler_path = '/content/drive/MyDrive/준희/Report/Files/StandardScaler_wo_env_dice.pkl'

In [23]:
### 모델 불러오기
model_path = '/content/drive/MyDrive/세은/model.pickle'
xgb_model = joblib.load(model_path)

In [64]:
### 기존 test 데이터로 하면 잘 되는데,,,,
pothole = pd.read_csv('/content/drive/MyDrive/DATA_완성/pothole_예측용.csv')
pothole['발생여부'] = np.where(pothole['포트홀 갯수'] >=1, 1, 0)

x = pothole[['차선수', '승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수', '배수등급', '경사도']]
x_test = x.iloc[8504:8520]

### DiCE 구성
continuous = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수']
d = dice_ml.Data(dataframe = x_test.assign(발생여부 = 0),  # dummy target
                 continuous_features = continuous,
                 outcome_name = "발생여부")

m = dice_ml.Model(model = xgb_model, backend = "sklearn")
exp = dice_ml.Dice(d, m)


### 예측이 1인 샘플 하나 골라서 counterfactual 생성
query_instance = x_test
while xgb_model.predict(query_instance)[0] != 1:
    query_instance = x_test.sample(n=1)
# 값을 바꿀 변수들만 지정
features_to_change = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령']

cf2 = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite",features_to_vary=features_to_change)
cf2.visualize_as_dataframe()

100%|██████████| 16/16 [07:13<00:00, 27.09s/it]

Query instance (original outcome : 1)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,0.018409,0.04493,0.223607,0.028154,0.20853,-0.503202,-0.419831,2.0,1.0,1



Diverse Counterfactual set (new outcome: 0)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,0.031984,-0.4335,-0.236145,-0.000416,-0.78748,-0.503202,-0.419831,2.0,1.0,0
1,6.0,0.031984,-0.4335,-0.236145,-0.000416,-0.78748,-0.503202,-0.419831,2.0,1.0,0


...

,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,0.107349,-0.023591,0.180708,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,0
1,6.0,0.107349,-0.023591,-0.261839,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,0
2,6.0,0.107349,-0.023591,0.182218,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,0
3,6.0,0.107349,-0.023591,0.174248,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,0
4,6.0,0.107349,-0.023591,0.122494,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,0


Query instance (original outcome : 1)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,0.107349,-0.023591,0.014501,0.08229,-0.34278,-0.503202,-0.419831,2.0,1.0,1



Diverse Counterfactual set (new outcome: 0)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,0.107349,-0.023591,0.014501,-0.022093,-0.34278,-0.503202,-0.419831,2.0,1.0,0
1,6.0,0.107349,-0.023591,-0.243211,0.082290,-0.34278,-0.503202,-0.419831,2.0,1.0,0
2,6.0,0.107349,-0.023591,0.098143,0.082290,-0.34278,-0.503202,-0.419831,2.0,1.0,0
3,6.0,0.107349,-0.023591,0.181089,0.082290,-0.34278,-0.503202,-0.419831,2.0,1.0,0
4,6.0,0.107349,-0.023591,0.083963,0.082290,-0.34278,-0.503202,-0.419831,2.0,1.0,0


In [52]:
### 기존 test 데이터로 하면 잘 되는데,,,,
pothole = pd.read_csv('/content/drive/MyDrive/DATA_완성/pothole_예측용.csv')
pothole['발생여부'] = np.where(pothole['포트홀 갯수'] >=1, 1, 0)

x = pothole[['차선수', '승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수', '배수등급', '경사도']]
x_test = x.iloc[8504:8585]

### DiCE 구성
continuous = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령', '인구 수']
d = dice_ml.Data(dataframe = x_test.assign(발생여부 = 0),  # dummy target
                 continuous_features = continuous,
                 outcome_name = "발생여부")

m = dice_ml.Model(model = xgb_model, backend = "sklearn")
exp = dice_ml.Dice(d, m)


### 예측이 1인 샘플 하나 골라서 counterfactual 생성
query_instance = x_test
while xgb_model.predict(query_instance)[0] != 1:
    query_instance = x_test.sample(n=1)
# 값을 바꿀 변수들만 지정
features_to_change = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령']

cf = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite",features_to_vary=features_to_change)
cf.visualize_as_dataframe()

  4%|▎         | 3/81 [00:01<00:27,  2.81it/s]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


 91%|█████████▏| 74/81 [28:42<00:49,  7.03s/it]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


100%|██████████| 81/81 [31:37<00:00, 23.43s/it]

...

,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,8.0,0.893040,-0.430816,0.68408,0.068316,-0.132429,-0.503202,-0.419831,4.0,2.0,0
1,8.0,0.115275,-0.337650,0.68408,0.581732,0.701229,-0.503202,-0.419831,4.0,2.0,0


Query instance (original outcome : 1)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,8.0,0.89304,0.427855,0.68408,0.86416,-0.520104,-0.503202,-0.419831,4.0,2.0,1



Diverse Counterfactual set (new outcome: 0)


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,8.0,0.320606,-0.117005,0.68408,0.501607,0.416585,-0.503202,-0.419831,4.0,2.0,0


In [78]:
# 기존 데이터
original_one = cf.cf_examples_list[5].test_instance_df
# 바뀐 데이터
changed_zero = cf.cf_examples_list[5].final_cfs_df

In [75]:
# 기존 데이터
original_one = cf.cf_examples_list[5].test_instance_df
# 바뀐 데이터
changed_zero = cf.cf_examples_list[5].final_cfs_df

original_one = inverse_scaling_boxcox(original_one, transformer_path, scaler_path)
changed_zero = inverse_scaling_boxcox(changed_zero, transformer_path, scaler_path)

In [76]:
display(original_one)
display(changed_zero)

,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,20131.000001,743.000001,2456.000012,23329.999997,0.137121,29.96128,169016.646453,약간불량,2-7%,1


,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도,발생여부
0,6.0,20524.538027,484.831871,2456.000001,31567.340417,0.106118,29.96128,169016.64706,약간불량,2-7%,0
1,6.0,20131.000001,495.870667,2250.090923,33690.123507,0.119051,29.96128,169016.64706,약간불량,2-7%,0
2,6.0,19206.273534,743.000001,2254.537737,32253.899707,0.109777,29.96128,169016.64706,약간불량,2-7%,0
3,6.0,20131.000001,555.496787,3486.346211,30415.481371,0.108937,29.96128,169016.64706,약간불량,2-7%,0
4,6.0,19206.273534,740.801458,2254.537737,32253.899707,0.109777,29.96128,169016.64706,약간불량,2-7%,0


In [77]:
original_one.to_csv('/content/drive/MyDrive/준희/Report/Files/original_one_pres.csv', encoding = 'utf-8-sig')
changed_zero.to_csv('/content/drive/MyDrive/준희/Report/Files/changed_zero_pres.csv', encoding = 'utf-8-sig')

In [9]:
pothole = pd.read_csv('/content/drive/MyDrive/DATA_완성/pothole_예측용.csv')
pothole['발생여부'] = np.where(pothole['포트홀 갯수'] >=1, 1, 0)

In [61]:
pothole[(pothole['행정동'] == '서초동') & (pothole['발생여부'] == 1) & (pothole['차선수'] == 6.0) & (pothole['배수등급'] == 2.0) & (pothole['경사도'] == 1.0) ]

,Unnamed: 0,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,...,발생일,인구 수,누적 강수량,평균 습도,1년 기온차,평균 일교차,배수등급,경사도,자치구_enc,발생여부
8486,8486,서초구,서초동,127.009400,37.483448,6.0,-0.542028,0.059023,0.253554,-0.446039,...,2023-06-07,-0.419831,0.706197,-1.007982,0.530717,1.774867,2.0,1.0,14,1
8490,8490,서초구,서초동,127.010842,37.484230,6.0,-0.539977,0.025367,0.320539,-0.435483,...,2023-06-24,-0.419831,0.706197,-0.945599,0.530717,0.819050,2.0,1.0,14,1
8491,8491,서초구,서초동,127.009991,37.484193,6.0,-0.539977,0.025367,0.320539,-0.435483,...,2023-07-17,-0.419831,0.706197,-0.570600,0.530717,-0.750415,2.0,1.0,14,1
8504,8504,서초구,서초동,127.025917,37.487587,6.0,0.018409,0.044930,0.223607,0.028154,...,2023-05-20,-0.419831,0.706197,-1.040852,0.530717,1.942164,2.0,1.0,14,1
8506,8506,서초구,서초동,127.022729,37.486767,6.0,0.018409,0.044930,0.223607,0.028154,...,2023-06-17,-0.419831,0.706197,-0.981892,0.530717,0.944159,2.0,1.0,14,1
8507,8507,서초구,서초동,127.022016,37.486329,6.0,0.018409,0.044930,0.223607,0.028154,...,2023-03-21,-0.419831,0.706197,-0.514491,0.530717,1.692917,2.0,1.0,14,1
8508,8508,서초구,서초동,127.027394,37.488014,6.0,0.018409,0.044930,0.223607,0.028154,...,2023-07-26,-0.419831,0.706197,-0.440999,0.530717,-1.356310,2.0,1.0,14,1
8509,8509,서초구,서초동,127.022310,37.486463,6.0,0.018409,0.044930,0.223607,0.028154,...,2023-01-28,-0.419831,0.706197,-0.007943,0.530717,-0.175408,2.0,1.0,14,1
8517,8517,서초구,서초동,127.013930,37.491574,6.0,0.051210,-0.466460,0.101047,0.026656,...,2023-06-28,-0.419831,0.706197,-0.907278,0.530717,0.805243,2.0,1.0,14,1
8523,8523,서초구,서초동,127.022007,37.486851,6.0,0.125716,0.132923,0.262822,0.130050,...,2023-06-17,-0.419831,0.706197,-0.981892,0.530717,0.944159,2.0,1.0,14,1


In [8]:
### 새로운 데이터 입력 받아서 하려고 하면 안 된다,,? 왜지??!??!?
new_x = pd.read_csv("/content/drive/MyDrive/준희/Report/Files/new_places_x_pres.csv", encoding = "utf-8-sig")
new_x.drop(['Unnamed: 0'], axis = 1, inplace = True)
original_one, changed_zero = dice(new_x, model_path, transformer_path, scaler_path)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:32:41] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 01 sec


UserConfigValidationException: No counterfactuals found for any of the query points! Kindly check your configuration.